- 날씨 데이터에서 평균온도 예측모델 만들기
- LGBM 사용하기

In [1]:
# 스파크 드라이버와 워커의 파이썬 버전이 다를 경우, 
# PYSPARK_PYTHON 와 PYSPARK_DRIVER_PYTHON 를 환경변수 지정 
# 예시 : .zshrc 파일에 아래 코드 입력
#           export PYSPARK_PYTHON=$HOME/.pyenv/versions/3.12.1/bin/python3.12
#           export PYSPARK_DRIVER_PYTHON=$HOME/.pyenv/versions/3.12.1/bin/python3.12
# 위의 설정을 해도 오류가 날 경우, 스크립트에 아래 코드 입력
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
# 스파크 세션 생성
import pyspark
from pyspark.sql import SparkSession

# SynapseML : 마이크로소프트에서 만든 스파크용 ML 라이브러리
# https://microsoft.github.io/SynapseML/
# config에서 synapseml 모듈 불러오기
spark = SparkSession.builder \
    .appName("Spark study - 231212") \
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.2") \
    .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven") \
    .getOrCreate()

In [ ]:
# 복잡한 JSON 파일을 읽을 때는 multiline 옵션을 쓰자

#df_bike = spark.read.option('multiline', 'true').json('file:///home/kjh/data/bike/bike.json')
df_final = spark.read.option('multiline', 'true').json('file:///home/kjh/data/bike/json_cycle_finale.json')

In [4]:
# 파일의 구조 확인
#df_bike.printSchema()
df_final.printSchema()

root
 |-- response: struct (nullable = true)
 |    |-- body: struct (nullable = true)
 |    |    |-- dataType: string (nullable = true)
 |    |    |-- items: struct (nullable = true)
 |    |    |    |-- item: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- avgCm10Te: string (nullable = true)
 |    |    |    |    |    |-- avgCm20Te: string (nullable = true)
 |    |    |    |    |    |-- avgCm30Te: string (nullable = true)
 |    |    |    |    |    |-- avgCm5Te: string (nullable = true)
 |    |    |    |    |    |-- avgLmac: string (nullable = true)
 |    |    |    |    |    |-- avgM05Te: string (nullable = true)
 |    |    |    |    |    |-- avgM10Te: string (nullable = true)
 |    |    |    |    |    |-- avgM15Te: string (nullable = true)
 |    |    |    |    |    |-- avgM30Te: string (nullable = true)
 |    |    |    |    |    |-- avgM50Te: string (nullable = true)
 |    |    |    |    |    |-- avgPa: string (nullabl

In [5]:
from pyspark.sql.functions import explode

# 중첩된 JSON 구조 접근 및 explode 함수 사용
#df_bike = df_bike.select(explode("useStatus.row").alias("items"))
df_final = df_final.select(explode("response.body.items.item").alias("items"))

In [6]:
#df_bike.show()
df_final.show()

+--------------------+
|               items|
+--------------------+
|{0.5, 1.9, 2.9, -...|
|{0.7, 1.7, 2.6, 0...|
|{1.0, 1.8, 2.7, 0...|
|{1.1, 1.9, 2.7, 0...|
|{1.0, 1.8, 2.6, 0...|
|{1.3, 1.9, 2.6, 1...|
|{3.8, 3.3, 3.6, 4...|
|{4.6, 4.5, 4.7, 4...|
|{2.9, 3.6, 4.3, 2...|
|{1.8, 2.8, 3.6, 0...|
|{1.6, 2.4, 3.2, 0...|
|{1.6, 2.3, 3.0, 1...|
|{1.1, 2.0, 2.9, 0...|
|{0.5, 1.6, 2.5, -...|
|{0.2, 1.2, 2.1, -...|
|{0.1, 1.0, 1.9, -...|
|{0.1, 1.0, 1.8, -...|
|{0.1, 1.0, 1.8, -...|
|{0.2, 1.0, 1.8, 0...|
|{0.5, 1.2, 1.9, 0...|
+--------------------+
only showing top 20 rows



In [7]:
# items 컬럼의 필드를 별도의 컬럼으로 펼침
df_final = df_final.select("items.*")

In [8]:
# 최종 데이터프레임 구조 확인 및 데이터 확인
df_final.printSchema()

root
 |-- avgCm10Te: string (nullable = true)
 |-- avgCm20Te: string (nullable = true)
 |-- avgCm30Te: string (nullable = true)
 |-- avgCm5Te: string (nullable = true)
 |-- avgLmac: string (nullable = true)
 |-- avgM05Te: string (nullable = true)
 |-- avgM10Te: string (nullable = true)
 |-- avgM15Te: string (nullable = true)
 |-- avgM30Te: string (nullable = true)
 |-- avgM50Te: string (nullable = true)
 |-- avgPa: string (nullable = true)
 |-- avgPs: string (nullable = true)
 |-- avgPv: string (nullable = true)
 |-- avgRhm: string (nullable = true)
 |-- avgTa: string (nullable = true)
 |-- avgTca: string (nullable = true)
 |-- avgTd: string (nullable = true)
 |-- avgTs: string (nullable = true)
 |-- avgWs: string (nullable = true)
 |-- ddMefs: string (nullable = true)
 |-- ddMefsHrmt: string (nullable = true)
 |-- ddMes: string (nullable = true)
 |-- ddMesHrmt: string (nullable = true)
 |-- hr1MaxIcsr: string (nullable = true)
 |-- hr1MaxIcsrHrmt: string (nullable = true)
 |-- hr1MaxR

In [9]:
print(df_final.columns)

['avgCm10Te', 'avgCm20Te', 'avgCm30Te', 'avgCm5Te', 'avgLmac', 'avgM05Te', 'avgM10Te', 'avgM15Te', 'avgM30Te', 'avgM50Te', 'avgPa', 'avgPs', 'avgPv', 'avgRhm', 'avgTa', 'avgTca', 'avgTd', 'avgTs', 'avgWs', 'ddMefs', 'ddMefsHrmt', 'ddMes', 'ddMesHrmt', 'hr1MaxIcsr', 'hr1MaxIcsrHrmt', 'hr1MaxRn', 'hr1MaxRnHrmt', 'hr24SumRws', 'iscs', 'maxInsWs', 'maxInsWsHrmt', 'maxInsWsWd', 'maxPs', 'maxPsHrmt', 'maxTa', 'maxTaHrmt', 'maxWd', 'maxWs', 'maxWsHrmt', 'maxWsWd', 'mi10MaxRn', 'mi10MaxRnHrmt', 'minPs', 'minPsHrmt', 'minRhm', 'minRhmHrmt', 'minTa', 'minTaHrmt', 'minTg', 'n99Rn', 'ssDur', 'stnId', 'stnNm', 'sumDpthFhsc', 'sumFogDur', 'sumGsr', 'sumLrgEv', 'sumRn', 'sumRnDur', 'sumSmlEv', 'sumSsHr', 'tm']


In [10]:
# 필요없어 보이는 컬럼 삭제
df_final = df_final.drop('iscs', 'stnNm')

In [11]:
df_final.show()

+---------+---------+---------+--------+-------+--------+--------+--------+--------+--------+------+------+-----+------+-----+------+-----+-----+-----+------+----------+-----+---------+----------+--------------+--------+------------+----------+--------+------------+----------+------+---------+-----+---------+-----+-----+---------+-------+---------+-------------+------+---------+------+----------+-----+---------+-----+-----+-----+-----+-----------+---------+------+--------+-----+--------+--------+-------+----------+
|avgCm10Te|avgCm20Te|avgCm30Te|avgCm5Te|avgLmac|avgM05Te|avgM10Te|avgM15Te|avgM30Te|avgM50Te| avgPa| avgPs|avgPv|avgRhm|avgTa|avgTca|avgTd|avgTs|avgWs|ddMefs|ddMefsHrmt|ddMes|ddMesHrmt|hr1MaxIcsr|hr1MaxIcsrHrmt|hr1MaxRn|hr1MaxRnHrmt|hr24SumRws|maxInsWs|maxInsWsHrmt|maxInsWsWd| maxPs|maxPsHrmt|maxTa|maxTaHrmt|maxWd|maxWs|maxWsHrmt|maxWsWd|mi10MaxRn|mi10MaxRnHrmt| minPs|minPsHrmt|minRhm|minRhmHrmt|minTa|minTaHrmt|minTg|n99Rn|ssDur|stnId|sumDpthFhsc|sumFogDur|sumGsr|sumLrgEv|su

In [12]:
# 컬럼의 형식변환
# tm 컬럼은 String으로 된 YYYY-MM-DD 날짜 형식. VectorAssembler 을 사용하기 위해서는 숫자로 변환 필요
# Date 로 먼저 변환한 뒤, Timestamp 형식으로 변환 => 최종 long 타입

from pyspark.sql.functions import col, unix_timestamp, to_date

df_fin = df_final

# 'tm' 컬럼을 날짜 타입으로 변환하고, 이를 Unix timestamp로 변환
df_fin = df_fin.withColumn('tm', unix_timestamp(to_date(col('tm'), 'yyyy-MM-dd')))

data_types = {
    "stnId": "int",  # 지점 번호는 정수
    "avgM15Te": "float",  # 지중온도는 소수점을 포함할 수 있으므로 float
    "n99Rn": "float",  # 강수량은 소수점을 포함할 수 있음
    "minPs": "float",  # 기압은 소수점을 포함할 수 있음
    "avgRhm": "float",  # 상대습도는 소수점을 포함할 수 있음
    "minRhmHrmt": "int",  # 시각은 정수로 처리
    "maxInsWsWd": "int",  # 풍향은 정수로 처리
    "avgTs": "float",  # 지면온도는 소수점을 포함할 수 있음
    "maxInsWsHrmt": "int",  # 시각은 정수로 처리
    "ddMesHrmt": "int",  # 시각은 정수로 처리
    "maxPsHrmt": "int",  # 시각은 정수로 처리
    "avgPv": "float",  # 증기압은 소수점을 포함할 수 있음
    "minRhm": "float",  # 상대습도는 소수점을 포함할 수 있음
    "sumSsHr": "float",  # 일조 시간은 소수점을 포함할 수 있음
    "ssDur": "float",  # 가조시간은 소수점을 포함할 수 있음
    "avgPs": "float",  # 해면기압은 소수점을 포함할 수 있음
    "maxWs": "float",  # 풍속은 소수점을 포함할 수 있음
    "avgCm5Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "minTg": "float",  # 초상온도는 소수점을 포함할 수 있음
    "maxWsWd": "int",  # 풍향은 정수로 처리
    "sumSmlEv": "float",  # 증발량은 소수점을 포함할 수 있음
    "avgTca": "float",  # 전운량은 소수점을 포함할 수 있음
    "hr1MaxIcsr": "float",  # 일사량은 소수점을 포함할 수 있음
    "avgTd": "float",  # 이슬점온도는 소수점을 포함할 수 있음
    "maxPs": "float",  # 해면기압은 소수점을 포함할 수 있음
    "avgCm20Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "ddMes": "float",  # 적설량은 소수점을 포함할 수 있음
    "minTa": "float",  # 기온은 소수점을 포함할 수 있음
    "minPsHrmt": "int",  # 시각은 정수로 처리
    "avgM50Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "maxTa": "float",  # 기온은 소수점을 포함할 수 있음
    "hr24SumRws": "int",  # 풍정합은 정수로 처리
    "avgM30Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "avgCm10Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "avgM05Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "hr1MaxIcsrHrmt": "int",  # 시각은 정수로 처리
    "maxInsWs": "float",  # 풍속은 소수점을 포함할 수 있음
    "avgTca": "float",  # 전운량은 소수점을 포함할 수 있음
    "avgCm30Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "avgM10Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "sumGsr": "float",  # 일사량은 소수점을 포함할 수 있음
    "maxWsHrmt": "int",  # 시각은 정수로 처리
    "avgPa": "float",  # 현지기압은 소수점을 포함할 수 있음
    "avgWs": "float",  # 풍속은 소수점을 포함할 수 있음
    "sumFogDur": "float",  # 안개 지속 시간은 소수점을 포함할 수 있음
    "sumLrgEv": "float",  # 증발량은 소수점을 포함할 수 있음
    "sumDpthFhsc": "float",  # 신적설은 소수점을 포함할 수 있음
    "ddMefs": "float",  # 적설량은 소수점을 포함할 수 있음
    "ddMefsHrmt": "int",  # 시각은 정수로 처리
    "sumRn": "float",  # 강수량은 소수점을 포함할 수 있음
    "hr1MaxRnHrmt": "int",  # 시각은 정수로 처리
    "hr1MaxRn": "float",  # 강수량은 소수점을 포함할 수 있음
    "mi10MaxRnHrmt": "int",  # 시각은 정수로 처리
    "mi10MaxRn": "float",  # 강수량은 소수점을 포함할 수 있음
    "avgTa": "float",  # 기온은 소수점을 포함할 수 있음
    "minTaHrmt": "int",  # 시각은 정수로 처리
    "maxTaHrmt": "int",  # 시각은 정수로 처리
    "maxWd": "int",  # 풍향은 정수로 처리
    "avgLmac": "float",  # 중하층운량은 소수점을 포함할 수 있음
    "sumRnDur": "float"
}

# 딕셔너리 형태의 컬럼 타입정보를 가지고 withColumn을 for문으로 돌림
for column, data_type in data_types.items():
    df_fin = df_fin.withColumn(column, col(column).cast(data_type))

In [13]:
df_fin.printSchema()

root
 |-- avgCm10Te: float (nullable = true)
 |-- avgCm20Te: float (nullable = true)
 |-- avgCm30Te: float (nullable = true)
 |-- avgCm5Te: float (nullable = true)
 |-- avgLmac: float (nullable = true)
 |-- avgM05Te: float (nullable = true)
 |-- avgM10Te: float (nullable = true)
 |-- avgM15Te: float (nullable = true)
 |-- avgM30Te: float (nullable = true)
 |-- avgM50Te: float (nullable = true)
 |-- avgPa: float (nullable = true)
 |-- avgPs: float (nullable = true)
 |-- avgPv: float (nullable = true)
 |-- avgRhm: float (nullable = true)
 |-- avgTa: float (nullable = true)
 |-- avgTca: float (nullable = true)
 |-- avgTd: float (nullable = true)
 |-- avgTs: float (nullable = true)
 |-- avgWs: float (nullable = true)
 |-- ddMefs: float (nullable = true)
 |-- ddMefsHrmt: integer (nullable = true)
 |-- ddMes: float (nullable = true)
 |-- ddMesHrmt: integer (nullable = true)
 |-- hr1MaxIcsr: float (nullable = true)
 |-- hr1MaxIcsrHrmt: integer (nullable = true)
 |-- hr1MaxRn: float (nullable 

In [14]:
df_fin.show()

+---------+---------+---------+--------+-------+--------+--------+--------+--------+--------+------+------+-----+------+-----+------+-----+-----+-----+------+----------+-----+---------+----------+--------------+--------+------------+----------+--------+------------+----------+------+---------+-----+---------+-----+-----+---------+-------+---------+-------------+------+---------+------+----------+-----+---------+-----+-----+-----+-----+-----------+---------+------+--------+-----+--------+--------+-------+----------+
|avgCm10Te|avgCm20Te|avgCm30Te|avgCm5Te|avgLmac|avgM05Te|avgM10Te|avgM15Te|avgM30Te|avgM50Te| avgPa| avgPs|avgPv|avgRhm|avgTa|avgTca|avgTd|avgTs|avgWs|ddMefs|ddMefsHrmt|ddMes|ddMesHrmt|hr1MaxIcsr|hr1MaxIcsrHrmt|hr1MaxRn|hr1MaxRnHrmt|hr24SumRws|maxInsWs|maxInsWsHrmt|maxInsWsWd| maxPs|maxPsHrmt|maxTa|maxTaHrmt|maxWd|maxWs|maxWsHrmt|maxWsWd|mi10MaxRn|mi10MaxRnHrmt| minPs|minPsHrmt|minRhm|minRhmHrmt|minTa|minTaHrmt|minTg|n99Rn|ssDur|stnId|sumDpthFhsc|sumFogDur|sumGsr|sumLrgEv|su

In [15]:
# 결측치 확인

from pyspark.sql.functions import col, count, when, isnull

# 각 컬럼에 대해 non-null 값을 계산
non_null_counts = df_fin.select([count(when(~isnull(c), c)).alias(c) for c in df_final.columns])

In [16]:
non_null_counts.show()

+---------+---------+---------+--------+-------+--------+--------+--------+--------+--------+-----+-----+-----+------+-----+------+-----+-----+-----+------+----------+-----+---------+----------+--------------+--------+------------+----------+--------+------------+----------+-----+---------+-----+---------+-----+-----+---------+-------+---------+-------------+-----+---------+------+----------+-----+---------+-----+-----+-----+-----+-----------+---------+------+--------+-----+--------+--------+-------+----+
|avgCm10Te|avgCm20Te|avgCm30Te|avgCm5Te|avgLmac|avgM05Te|avgM10Te|avgM15Te|avgM30Te|avgM50Te|avgPa|avgPs|avgPv|avgRhm|avgTa|avgTca|avgTd|avgTs|avgWs|ddMefs|ddMefsHrmt|ddMes|ddMesHrmt|hr1MaxIcsr|hr1MaxIcsrHrmt|hr1MaxRn|hr1MaxRnHrmt|hr24SumRws|maxInsWs|maxInsWsHrmt|maxInsWsWd|maxPs|maxPsHrmt|maxTa|maxTaHrmt|maxWd|maxWs|maxWsHrmt|maxWsWd|mi10MaxRn|mi10MaxRnHrmt|minPs|minPsHrmt|minRhm|minRhmHrmt|minTa|minTaHrmt|minTg|n99Rn|ssDur|stnId|sumDpthFhsc|sumFogDur|sumGsr|sumLrgEv|sumRn|sumRnDur|s

In [17]:
# 결측치 컬럼 제거
drop_cols = ['ddMefs', 'ddMefsHrmt', 'ddMes', 'ddMesHrmt', 'hr1MaxRn', 'hr1MaxRnHrmt', 'mi10MaxRn', 'mi10MaxRnHrmt', 'n99Rn', 'sumDpthFhsc', 'sumFogDur', 'sumRn', 'sumRnDur']
df = df_fin.drop(*drop_cols)

In [18]:
df.show()

+---------+---------+---------+--------+-------+--------+--------+--------+--------+--------+------+------+-----+------+-----+------+-----+-----+-----+----------+--------------+----------+--------+------------+----------+------+---------+-----+---------+-----+-----+---------+-------+------+---------+------+----------+-----+---------+-----+-----+-----+------+--------+--------+-------+----------+
|avgCm10Te|avgCm20Te|avgCm30Te|avgCm5Te|avgLmac|avgM05Te|avgM10Te|avgM15Te|avgM30Te|avgM50Te| avgPa| avgPs|avgPv|avgRhm|avgTa|avgTca|avgTd|avgTs|avgWs|hr1MaxIcsr|hr1MaxIcsrHrmt|hr24SumRws|maxInsWs|maxInsWsHrmt|maxInsWsWd| maxPs|maxPsHrmt|maxTa|maxTaHrmt|maxWd|maxWs|maxWsHrmt|maxWsWd| minPs|minPsHrmt|minRhm|minRhmHrmt|minTa|minTaHrmt|minTg|ssDur|stnId|sumGsr|sumLrgEv|sumSmlEv|sumSsHr|        tm|
+---------+---------+---------+--------+-------+--------+--------+--------+--------+--------+------+------+-----+------+-----+------+-----+-----+-----+----------+--------------+----------+--------+-------

In [19]:
# 결측 데이터 있는 행 제거
df = df.dropna()

In [20]:
# # 범주형 컬럼 카테고리 인코딩하는 예제
# # 기본적으로 라벨인코딩 -> 원핫인코딩 순서를 거쳐야 한다.
# from pyspark.ml.feature import StringIndexer

# indexer = StringIndexer(inputCol="stnNm", outputCol="indexed_stnNm")
# indexed = indexer.fit(df).transform(df)

# from pyspark.ml.feature import OneHotEncoder

# encoder = OneHotEncoder(inputCols=["indexed_stnNm"], outputCols=["encoded_stnNm"])
# encoded = encoder.fit(indexed).transform(indexed)

In [21]:
from pyspark.ml.feature import VectorAssembler

In [22]:
# 데이터프레임의 모든 컬럼을 특성으로 사용하고자 할 때
features = df.columns
features.remove('avgTa')  # 라벨 컬럼 제외

In [23]:
# VectorAssembler 생성 및 변환
assembler = VectorAssembler(inputCols=features, outputCol="features_vector")
df_vector = assembler.transform(df)

In [24]:
df_vector.show()

+---------+---------+---------+--------+-------+--------+--------+--------+--------+--------+------+------+-----+------+-----+------+-----+-----+-----+----------+--------------+----------+--------+------------+----------+------+---------+-----+---------+-----+-----+---------+-------+------+---------+------+----------+-----+---------+-----+-----+-----+------+--------+--------+-------+----------+--------------------+
|avgCm10Te|avgCm20Te|avgCm30Te|avgCm5Te|avgLmac|avgM05Te|avgM10Te|avgM15Te|avgM30Te|avgM50Te| avgPa| avgPs|avgPv|avgRhm|avgTa|avgTca|avgTd|avgTs|avgWs|hr1MaxIcsr|hr1MaxIcsrHrmt|hr24SumRws|maxInsWs|maxInsWsHrmt|maxInsWsWd| maxPs|maxPsHrmt|maxTa|maxTaHrmt|maxWd|maxWs|maxWsHrmt|maxWsWd| minPs|minPsHrmt|minRhm|minRhmHrmt|minTa|minTaHrmt|minTg|ssDur|stnId|sumGsr|sumLrgEv|sumSmlEv|sumSsHr|        tm|     features_vector|
+---------+---------+---------+--------+-------+--------+--------+--------+--------+--------+------+------+-----+------+-----+------+-----+-----+-----+----------+

In [25]:
# train, test 쪼개기
train, test = df_vector.randomSplit([0.85, 0.15], seed=42)

In [26]:
from synapse.ml.lightgbm import LightGBMRegressor

In [27]:
# 모델 생성 및 훈련
lgbm = LightGBMRegressor(objective='regression',
                            featuresCol='features_vector',
                            labelCol='avgTa',
                            alpha=0.3,
                            learningRate=0.3,
                            numIterations=100,
                            numLeaves=31)

model = lgbm.fit(train)

[LightGBM] [Info] Saving data reference to binary buffer


In [28]:
# 예측
pred = model.transform(test)

In [29]:
# predictionCol 확인
model.getPredictionCol()

'prediction'

In [30]:
from pyspark.ml.evaluation import RegressionEvaluator

In [31]:
# 검증
evaluator = RegressionEvaluator(labelCol="avgTa", predictionCol="prediction", metricName="rmse")

In [32]:
rmse = evaluator.evaluate(pred)

In [33]:
print(rmse)

0.5688844324933624
